In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'CLIENTNUM'], axis=1)
cols = df.columns
num_data = list(df._get_numeric_data().columns)
cat_data = list(set(cols) - set(num_data))
num_data.remove('Dependent_count')
num_data.remove('Total_Relationship_Count')
num_data.remove('Months_Inactive_12_mon')
num_data.remove('Contacts_Count_12_mon')
cat_data.append('Dependent_count')
cat_data.append('Total_Relationship_Count')
cat_data.append('Months_Inactive_12_mon')
cat_data.append('Contacts_Count_12_mon')
num_data_plot = num_data
cat_data_plot = cat_data

df = df.loc[df['Card_Category'] != 'Platinum']
df = df.loc[df['Total_Ct_Chng_Q4_Q1'] <= 1.6]
df = df.loc[df['Months_Inactive_12_mon'] >= 1]
df = df.loc[df['Months_Inactive_12_mon'] < 6]
df = df.loc[df['Customer_Age'] <= 66]
df = df.loc[df['Total_Amt_Chng_Q4_Q1'] <= 1.5]
df['Credit_Limit'] = np.log10(df['Credit_Limit'])
fitted_data, fitted_lambda = stats.boxcox(df['Avg_Open_To_Buy'])
df['Avg_Open_To_Buy'] = fitted_data
df['Total_Trans_Amt'] = np.log10(df['Total_Trans_Amt'])
df['Avg_Utilization_Ratio'] = (df['Avg_Utilization_Ratio'])**(1/2)
y = df['Attrition_Flag']
x = df.drop(['Attrition_Flag'], axis=1)
x = pd.get_dummies(x)
y = y.replace(['Existing Customer', 'Attrited Customer'], [1, 0])

In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=25)
x_new = selector.fit(x,y)

col = selector.get_support(indices=True)
x_new = x.iloc[:,col]
x_new.head()

,Customer_Age,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,...,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Unknown,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$120K +,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_Less than $40K
5,44,2,3,1,2,3.603144,1247,1.376,3.036629,24,...,0,0,0,1,0,0,0,1,0,0
10,42,5,5,3,2,3.829175,1467,0.831,3.079543,42,...,0,0,0,0,0,1,1,0,0,0
11,65,1,6,2,3,3.958803,1587,1.433,3.118595,26,...,0,0,1,1,0,0,0,1,0,0
14,57,2,5,2,2,3.386677,680,1.190,3.195900,29,...,0,0,0,1,0,0,0,0,0,1
19,45,2,6,1,2,4.160469,1157,0.966,3.081707,21,...,0,0,0,1,0,0,0,0,0,0


In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x = x_new.values
y = y.values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x_scaled, y, test_size = 0.2, random_state = 1)


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


parameters = {'C':[0.01, 0.1, 1, 10, 100],
                'solver':('newton-cg', 'liblinear', 'sag', 'saga')}
logisReg = LogisticRegression()
clf = GridSearchCV(logisReg, parameters, scoring='f1')
clf.fit(x_train, y_train)

print(clf.best_params_)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

c:\Users\thana\Desktop\TheShit\FRA503\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
c:\Users\thana\Desktop\TheShit\FRA503\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
c:\Users\thana\Desktop\TheShit\FRA503\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
c:\Users\thana\Desktop\TheShit\FRA503\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
c:\Users\thana\Desktop\TheShit\FRA503\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


{'C': 100, 'solver': 'newton-cg'}
              precision    recall  f1-score   support

           0       0.78      0.62      0.69       317
           1       0.93      0.97      0.95      1641

    accuracy                           0.91      1958
   macro avg       0.85      0.79      0.82      1958
weighted avg       0.91      0.91      0.91      1958

